<img src="Bilder/ost_logo.png" width="240"  align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik/systemtechnik/ice-institut-fuer-computational-engineering/"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/U08/ANN_08_XAI_ResNet50V2_SOLUTION_pl.ipynb)

# Explainable AI: Visualizing what convnets learn


Oft heisst es, Deep-Learning-Modelle seien »Blackboxes«, die Repräsentationen in einer Form erlernen, die schwer zu extrahieren und für Menschen kaum verständlich sind. 
- Für einige Arten von Deep-Learning-Modellen mag das teilweise stimmen, auf CNNs trifft es jedoch definitiv nicht zu. 
- Die von CNNs erlernten Repräsentationen sind in hohem Mass für Visualisierungen geeignet, und zwar vor allem deshalb, weil es sich um Repräsentationen visueller Konzepte handelt. Seit 2013 wurde eine Vielzahl von Verfahren zur Visualisierung und Interpretation dieser Repräsentationen entwickelt. Wir werden nicht alle diese Verfahren betrachten, aber die drei verständlichsten und nützlichsten erörtern:

Abgesehen von der Befürchtung, dass böse künstliche Intelligenzen die Welt übernehmen werden, kann das Gebiet der künstlichen Intelligenz für Außenstehende entmutigend sein. Facebooks Direktor für künstliche Intelligenz, **Yann LeCun**, verwendet die Analogie, dass KI eine Blackbox mit einer Million Knöpfen ist; das Innenleben ist den meisten ein Rätsel. Aber jetzt haben wir einen Blick hinein geworfen.

**Adam Harley**, Masterstudent an der Ryerson University, hat eine interaktive Visualisierung erstellt, die erklärt, wie ein neuronales Faltungsnetz, eine Art Programm für künstliche Intelligenz, das zur Analyse von Bildern verwendet wird, intern funktioniert.

[Visualization of a CNN](http://www.cs.cmu.edu/~aharley/nn_vis/cnn/3d.html)

Visualisierung der zwischenliegenden Ausgaben (zwischenliegende Aktivierungen) – Dieses Verfahren ermöglicht es, zu verstehen, wie aufeinanderfolgende Layer ihre Eingaben transformieren, und vermittelt eine Vorstellung von der Bedeutung
der einzelnen CNN-Filter. 

* **Visualisierung der CNN-Filter** – Dieses Verfahren ermöglicht es, genau zu verstehen, für welche visuellen Muster oder Konzepte die CNN-Filter empfänglich sind.
* **Visualisierung der Heatmaps der Klassenaktivierung als Bild** – Dieses Verfahren ermöglicht es, zu verstehen, welche Teile eines Bilds als zu einer bestimmten Klasse zugehörig erkannt wurden. Auf diese Weise können die Positionen von Objekten in Bildern ermittelt werden.

## Einführung

In diesem Beispiel untersuchen wir, welche Art von visuellen Mustern Bildklassifikationsmodelle
lernen. Wir werden das Modell "ResNet50V2" verwenden, das auf dem ImageNet-Datensatz trainiert wurde.

Unser Verfahren ist einfach: Wir werden Eingabebilder erstellen, die die Aktivierung bestimmter Filter in einer
bestimmten Filtern in einer Zielschicht (die irgendwo in der Mitte des Modells ausgewählt wird: Schicht
`conv3_block4_out`). Solche Bilder stellen eine Visualisierung des
Muster, auf das der Filter reagiert.



## Setup

- Ziel des Codes: Visualisierung der Filter einer bestimmten Schicht des vortrainierten ResNet50-Modells.
Hauptschritte:
    - Laden des ResNet50-Modells mit vortrainierten Gewichten.
    - Ausgabe der Modellstruktur zur Identifikation der Zielschicht.
    - Definition einer Lightning-Klasse zur Kapselung des Modells.
    - Extraktion und Visualisierung der Filter der Zielschicht.

In [ ]:
import torch
import numpy as np
import lightning as L
import torch.nn as nn

from torchvision import models
from torchsummary import summary
from PIL import Image as PILImage
import matplotlib.pyplot as plt
from IPython.display import Image, display

# Laden des ResNet50-Modells mit vortrainierten ImageNet-Gewichten
model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)

# Ausgabe der Namen aller Schichten im Modell
print("Layer names in the model:")
for name, module in model.named_modules():
    print(name)

# Die Dimensionen des Eingabebildes
img_width = 180
img_height = 180

# Zielschicht: Wir werden die Filter aus dieser Schicht visualisieren.
# Sie können den Namen der Schicht ändern, indem Sie die Ausgabe von `model.named_modules()` überprüfen.
layer_name = "layer4.2.conv3"


# Definition einer Lightning-Modul-Klasse für ResNet50
class ResNet50V2Lightning(L.LightningModule):
    def __init__(self):
        super(ResNet50V2Lightning, self).__init__()
        # Laden des vortrainierten ResNet50-Modells
        self.model = models.resnet50(pretrained=True)
        # Setzen des Modells in den Evaluierungsmodus
        self.model.eval()

    def forward(self, x):
        # Vorwärtsdurchlauf durch das Modell
        return self.model(x)

    def visualize_filters(self):
        # Funktion zur Visualisierung der Filter aus der Zielschicht
        # Abrufen der Zielschicht aus dem Modell
        target_layer = dict([*self.model.named_modules()])[layer_name]
        # Abrufen der Gewichtsdaten der Zielschicht
        filters = target_layer.weight.data.cpu().numpy()
        return filters


# Beispielverwendung
# Instanziieren des Lightning-Modells
model = ResNet50V2Lightning()

# Visualisieren der Filter aus der Zielschicht
filters = model.visualize_filters()

# Ausgabe der Form der Filter (z. B. Anzahl der Filter, Dimensionen)
print(filters.shape)


## (a) Erstellen eines Modells zur Merkmalsextraktion

- Ziel: Anpassung eines vortrainierten ResNet50-Modells für Trainingszwecke.
Hauptschritte:
    - Laden des vortrainierten ResNet50-Modells mit ImageNet-Gewichten.
    - Entfernen der letzten Schicht (fc), um das Modell als Feature-Extractor zu verwenden.
    - Definition eines Trainingsschritts mit Cross-Entropy-Verlust.
    - Konfiguration des Adam-Optimierers.

In [ ]:
# Definition einer Lightning-Modul-Klasse für ResNet50
class ResNet50V2Lightning(L.LightningModule):
    def __init__(self):
        super(ResNet50V2Lightning, self).__init__()
        # Laden des vortrainierten ResNet50-Modells mit ImageNet-Gewichten
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # Entfernen der voll verbundenen Schicht (fully connected layer), um das Modell als Feature-Extractor zu verwenden
        self.model.fc = nn.Identity()

    def forward(self, x):
        # Vorwärtsdurchlauf durch das Modell
        return self.model(x)

    def training_step(self, batch, batch_idx):
        # Trainingsschritt: Berechnung des Verlusts (Cross-Entropy) zwischen den Vorhersagen und den Labels
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        # Konfiguration des Optimierers (Adam-Optimierer mit Lernrate 0.001)
        return torch.optim.Adam(self.parameters(), lr=1e-3)


# Instanziieren des Lightning-Modells
model = ResNet50V2Lightning()


Überprüfung der GPU-Verfügbarkeit: Es wird geprüft, ob eine GPU verfügbar ist. Falls ja, wird das Gerät auf cuda gesetzt, andernfalls auf cpu.

Modell auf das Gerät verschieben: Das Modell wird auf das ausgewählte Gerät (GPU oder CPU) verschoben, damit die Berechnungen entsprechend dort ausgeführt werden können.

Modellzusammenfassung ausgeben: Mit der Funktion summary wird eine Übersicht des Modells erstellt. Diese zeigt die Architektur des Modells, die Anzahl der Parameter und die Dimensionen der Zwischenausgaben. Die Eingabegrösse wird dabei als (3, 224, 224) angegeben, was einem Bild mit 3 Farbkanälen (RGB) und einer Auflösung von 224x224 Pixeln entspricht.

In [ ]:
# Überprüfen, ob eine GPU verfügbar ist, und das Gerät entsprechend festlegen
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Verschieben des Modells auf das ausgewählte Gerät (GPU oder CPU)
model = model.to(device)

# Ausgabe einer Zusammenfassung des Modells
# Die Funktion `summary` zeigt die Architektur des Modells, die Anzahl der Parameter und die Dimensionen der Zwischenausgaben
summary(model, (3, 224, 224))  # Eingabegrösse: 3 Kanäle (RGB), 224x224 Pixel


In [ ]:
# Ausgabe der Architektur des Modells
# Dies zeigt die Struktur des ResNet50-Modells, einschliesslich aller Schichten und ihrer Parameter.
print(model.model)


Wie hier zu sehen ist gibt es vier Layers

## (b) Erstellen eines Modells zur Merkmalsextraktion

- Das Modell basiert auf ResNet50 und wurde so modifiziert, dass es als Feature-Extractor verwendet werden kann.
- Es ermöglicht die Extraktion der Aktivierungen einer bestimmten Zielschicht (z. B. "layer1").
- Der Code zeigt, wie das Modell mit einer Dummy-Eingabe verwendet werden kann, um die Form der Aktivierungen zu überprüfen.

In [ ]:
class ResNet50V2Lightning(L.LightningModule):
    def __init__(self, target_layer):
        super(ResNet50V2Lightning, self).__init__()
        # Laden des vortrainierten ResNet50-Modells mit ImageNet-Gewichten
        self.model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V2)
        # Entfernen der voll verbundenen Schicht (fully connected layer), um das Modell als Feature-Extractor zu verwenden
        self.model.fc = nn.Identity()
        # Speichern des Namens der Zielschicht, deren Aktivierungen extrahiert werden sollen
        self.target_layer = target_layer

    def forward(self, x):
        # Vorwärtsdurchlauf durch das Modell
        for name, layer in self.model.named_children():
            # Anwenden der aktuellen Schicht auf die Eingabe
            x = layer(x)
            # Überprüfen, ob die aktuelle Schicht die Zielschicht ist
            if name == self.target_layer:
                # Rückgabe der Aktivierungen der Zielschicht
                return x
        # Rückgabe der endgültigen Ausgabe, falls die Zielschicht nicht gefunden wurde
        return x

    def training_step(self, batch, batch_idx):
        # Trainingsschritt: Berechnung des Verlusts (Cross-Entropy) zwischen den Vorhersagen und den Labels
        x, y = batch
        y_hat = self(x)
        loss = nn.functional.cross_entropy(y_hat, y)
        return loss

    def configure_optimizers(self):
        # Konfiguration des Optimierers (Adam-Optimierer mit Lernrate 0.001)
        return torch.optim.Adam(self.parameters(), lr=1e-3)


# Definieren des Namens der Zielschicht
target_layer = "layer1"  # Beispiel: Zielschicht ist "layer1"
# target_layer = "layer2"  # Alternativ: Zielschicht ist "layer2"
# target_layer = "layer3"  # Alternativ: Zielschicht ist "layer3"
# target_layer = "layer4"  # Alternativ: Zielschicht ist "layer4"

# Instanziieren des Modells mit der Zielschicht
model = ResNet50V2Lightning(target_layer=target_layer)

# Beispielverwendung mit einer Dummy-Eingabe
dummy_input = torch.randn(
    1, 3, 224, 224
)  # Dummy-Bild mit der Grösse 224x224 und 3 Kanälen (RGB)
output = model(dummy_input)  # Abrufen der Aktivierungen der Zielschicht
print(output.shape)  # Ausgabe der Form der Aktivierungen

# Aktivierungen in ein numpy-Array konvertieren
activations = output.detach().cpu().numpy()


# Plotten der Aktivierungen
# Wir nehmen an, dass die Aktivierungen die Form (Batch, Channels, Height, Width) haben
num_filters = activations.shape[1]  # Anzahl der Filter (Kanäle)
fig, axes = plt.subplots(
    1, min(num_filters, 8), figsize=(20, 5)
)  # Zeige bis zu 8 Filter

for i, ax in enumerate(axes):
    if i >= num_filters:
        break
    # Zeige den i-ten Filter (wir nehmen die erste Batch-Dimension)
    ax.imshow(activations[0, i, :, :], cmap="viridis")
    ax.axis("off")
    ax.set_title(f"Filter {i}")

plt.show()


## (c) Einrichten des Gradientenanstiegsprozesses

- Die Funktion ``compute_loss`` berechnet den Mittelwert der Aktivierung eines bestimmten Filters in einer Zielschicht des Modells.
- Sie ignoriert Randpixel, um Artefakte zu vermeiden.
- Der Verlustwert gibt an, wie stark der ausgewählte Filter auf die Eingabe reagiert.
- In der Beispielverwendung wird ein Dummy-Bild verwendet, um den Verlust für den Filter mit Index 0 zu berechnen.

In [ ]:
def compute_loss(input_image, filter_index, model):
    # Verschieben des Eingabebildes auf dasselbe Gerät wie das Modell
    input_image = input_image.to(next(model.parameters()).device)

    # Abrufen der Aktivierung aus dem Modell
    activation = model(input_image)

    # Vermeidung von Randartefakten, indem nur Nicht-Randpixel in den Verlust einbezogen werden
    filter_activation = activation[:, filter_index, 2:-2, 2:-2]

    # Berechnung des Mittelwerts der Aktivierung als Verlust
    return torch.mean(filter_activation)


# Beispielverwendung mit einer Dummy-Eingabe und einem Filterindex
dummy_input = torch.randn(
    1, 3, 224, 224
)  # Dummy-Eingabebild mit der Grösse 224x224 und 3 Kanälen (RGB)
filter_index = 0  # Beispiel-Filterindex
loss = compute_loss(dummy_input, filter_index, model)  # Berechnung des Verlusts
print(loss)  # Ausgabe des Verlusts


Die Funktion ``gradient_ascent_step`` optimiert ein Eingabebild, um die Aktivierung eines bestimmten Filters in einer Zielschicht zu maximieren.
Dies wird durch Gradientenberechnung und Aktualisierung des Bildes erreicht.
In der Beispielverwendung wird ein zufälliges Dummy-Bild optimiert, um die Aktivierung des Filters mit Index 0 zu maximieren.
Solche Techniken werden oft verwendet, um zu visualisieren, welche Muster ein neuronales Netzwerk gelernt hat.

In [ ]:
def gradient_ascent_step(img, filter_index, learning_rate, model):
    # Setzen des Eingabebildes auf "requires_grad", um Gradienten berechnen zu können
    img.requires_grad = True
    # Initialisieren des Optimierers (SGD) mit dem Eingabebild und der Lernrate
    optimizer = torch.optim.SGD([img], lr=learning_rate)

    # Zurücksetzen der Gradienten des Optimierers
    optimizer.zero_grad()
    # Berechnung des Verlusts für den angegebenen Filterindex
    loss = compute_loss(img, filter_index, model)
    # Rückwärtsdurchlauf zur Berechnung der Gradienten
    loss.backward()

    # Normalisieren der Gradienten, um stabile Updates zu gewährleisten
    grads = img.grad / (torch.sqrt(torch.mean(img.grad**2)) + 1e-5)

    # Aktualisieren des Bildes basierend auf den Gradienten und der Lernrate
    img.data += learning_rate * grads.data

    # Rückgabe des Verlusts und des aktualisierten Bildes
    return loss, img


# Beispielverwendung mit einer Dummy-Eingabe und einem Filterindex
dummy_input = torch.randn(1, 3, 224, 224, requires_grad=True)  # Dummy-Eingabebild
filter_index = 0  # Beispiel-Filterindex
learning_rate = 0.1  # Beispiel-Lernrate

# Ausführen eines Gradientenanstiegsschritts
loss, updated_img = gradient_ascent_step(
    dummy_input, filter_index, learning_rate, model
)
print(loss)  # Ausgabe des Verlusts


## (d) Einrichten der End-to-End-Filtervisualisierungsschleife

- Der Code visualisiert, welche Eingaben einen bestimmten Filter in einem neuronalen Netzwerk (z. B. ResNet50) aktivieren.
- Ein zufälliges Bild wird iterativ optimiert, um die Aktivierung eines bestimmten Filters zu maximieren.
- Nach der Optimierung wird das Bild dekodiert und kann visualisiert werden, um zu verstehen, welche Muster der Filter gelernt hat.
- Dies ist eine Technik der Explainable AI (XAI), um die Funktionsweise von neuronalen Netzwerken besser zu verstehen.



In [ ]:
# Überprüfen, ob eine GPU verfügbar ist
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def initialize_image(img_width, img_height):
    # Wir starten mit einem grauen Bild mit etwas zufälligem Rauschen
    img = torch.rand((1, 3, img_width, img_height), device=device)
    # ResNet50V2 erwartet Eingaben im Bereich [-1, +1].
    # Hier skalieren wir unsere zufälligen Eingaben auf den Bereich [-0.125, +0.125]
    return (img - 0.5) * 0.25


def deprocess_image(img):
    # Normalisieren des Arrays: Zentrieren auf 0 und Sicherstellen einer Varianz von 0.15
    img -= img.mean()
    img /= img.std() + 1e-5
    img *= 0.15

    # Zuschneiden des Bildes (Center Crop)
    img = img[:, 25:-25, 25:-25]

    # Begrenzen auf den Bereich [0, 1]
    img += 0.5
    img = torch.clamp(img, 0, 1)

    # Konvertieren in ein RGB-Array
    img *= 255
    img = torch.clamp(img, 0, 255).byte().cpu().numpy().transpose(1, 2, 0)
    return img


def visualize_filter(filter_index, model, img_width=224, img_height=224):
    print("Visualisiere Filterindex:", filter_index)
    # Wir führen den Gradientenanstieg für 30 Schritte aus
    iterations = 30
    learning_rate = 10.0
    img = initialize_image(img_width, img_height)
    for iteration in range(iterations):
        # Gradientenanstiegsschritt ausführen
        loss, img = gradient_ascent_step(img, filter_index, learning_rate, model)

    # Dekodieren des resultierenden Eingabebildes
    img = deprocess_image(img.detach().cpu()[0])
    return loss, img


# Verschieben des Modells auf die GPU, falls verfügbar
model = model.to(device)


Probieren wir es mit Filter 0 in der Zielebene aus:

In [ ]:
def save_img(img, path):
    # Konvertiert den Tensor in ein PIL-Bild und speichert es
    pil_img = PILImage.fromarray(img)
    pil_img.save(path)


# Beispielverwendung
# Visualisiert den Filter mit Index 0 und speichert das resultierende Bild
loss, img = visualize_filter(0, model)
save_img(img, "0.png")  # Speichert das Bild unter dem Namen "0.png"
display(Image(filename="0.png"))  # Zeigt das gespeicherte Bild an


## (e) Visualisierung der ersten 64 Filter in der Zielebene

Erstellen wir nun ein 8x8-Gitter mit den ersten 64 Filtern
in der Zielebene, um ein Gefühl für die Bandbreite
der verschiedenen visuellen Muster zu bekommen, die das Modell gelernt hat.



In [ ]:
def save_img(img, path):
    # Konvertiert den Tensor in ein PIL-Bild und speichert es
    pil_img = PILImage.fromarray(img)
    pil_img.save(path)


# Berechnung der Eingabebilder, die die Aktivierungen der ersten 64 Filter
# in unserer Zielschicht maximieren
all_imgs = []
for filter_index in range(64):
    # Visualisierung des Filters mit dem aktuellen Index
    # und Hinzufügen des resultierenden Bildes zur Liste
    loss, img = visualize_filter(filter_index, model)
    all_imgs.append(img)

# Erstellen eines schwarzen Bildes mit genügend Platz für
# unsere 8 x 8 Filter mit einer Grösse von 128 x 128 Pixeln und einem Rand von 5 Pixeln dazwischen
margin = 5  # Rand zwischen den Bildern
n = 8  # Anzahl der Bilder pro Zeile und Spalte
cropped_width = 224 - 25 * 2  # Breite des zugeschnittenen Bildes
cropped_height = 224 - 25 * 2  # Höhe des zugeschnittenen Bildes
width = (
    n * cropped_width + (n - 1) * margin
)  # Gesamtbreite des zusammengesetzten Bildes
height = (
    n * cropped_height + (n - 1) * margin
)  # Gesamthöhe des zusammengesetzten Bildes
stitched_filters = np.zeros(
    (height, width, 3), dtype=np.uint8
)  # Initialisierung des schwarzen Bildes

# Füllen des zusammengesetzten Bildes mit unseren gespeicherten Filtern
for i in range(n):
    for j in range(n):
        img = all_imgs[i * n + j]  # Abrufen des Bildes für den aktuellen Filter
        stitched_filters[
            (cropped_height + margin) * i : (cropped_height + margin) * i
            + cropped_height,
            (cropped_width + margin) * j : (cropped_width + margin) * j + cropped_width,
            :,
        ] = img  # Platzieren des Bildes im zusammengesetzten Bild

# Speichern und Anzeigen des zusammengesetzten Bildes mit den Filtern
save_img(stitched_filters, "stitched_filters.png")  # Speichern des Bildes
display(Image(filename="stitched_filters.png"))  # Anzeigen des gespeicherten Bildes


### Zusammenfassung
- Das Notebook zeigt, wie CNN-Filter visualisiert werden können, um die erlernten Muster eines Modells zu interpretieren.
- Es kombiniert Techniken wie Gradientenberechnung, Bildoptimierung und Visualisierung.
- Die Ergebnisse helfen, die Funktionsweise von CNNs besser zu verstehen und sind ein Beispiel für Explainable AI (XAI).